In [ ]:
pip install pydealer

In [ ]:
import pydealer
import random

In [ ]:
deck = pydealer.Deck()
deck.shuffle()

user_hand = deck.deal(7)
agent_hand = deck.deal(7)

# converting the hand to a list of only ranks
user_hand_rank = []
agent_hand_rank = []


for i in user_hand:
  user_hand_rank.append(i.value)

for i in agent_hand:
  agent_hand_rank.append(i.value)

In [ ]:
print(len(user_hand))

7


In [ ]:
def call(isUser, hand, hand_rank ,isFirstRound, roundCardRank):

  # list of all possible ranks
  ranks = ['2', '3', '4', '5', '6', '7', '8', '9', '10', 'Jack', 'Queen', 'King', 'Ace']
  option = 0

  # if user is playing the first round
  if(isUser and isFirstRound):
    card_rank = input("\nEnter the rank of the card (caps): ")

    while card_rank not in ranks:
      print("\nWrong Card Rank")
      print("\nAvailable Ranks -> ['2', '3', '4', '5', '6', '7', '8', '9', '10', 'Jack', 'Queen', 'King', 'Ace']")
      card_rank = input()

  # if agent is playing the first round - Honesty or Bluff
  elif(not isUser and isFirstRound):
    option = random.randint(0,1)

    # agent choses to be honest
    if option == 0:
      card_rank = max(hand_rank, key=hand_rank.count)

    # agent choses to bluff
    else:
      card_rank = random.choice(tuple(ranks))

  # if agent/user is playing the consecutive rounds
  else:
    card_rank = roundCardRank

  # if user is playing
  if(isUser):
    total_selected_cards = safe_int_input("\nEnter the total number to call: ")

    while total_selected_cards <= 0:
      print("\nNumber can't be less than 0")
      total_selected_cards = safe_int_input("\nEnter the total number to call: ")


    selected_cards_index = []
    selected_cards_name = []



    while len(selected_cards_name) != total_selected_cards:
      selected_index = safe_int_input("\nEnter the index of the cards you want to chose for the call: ")
      if(selected_index < 0 or selected_index >= len(hand) or selected_index in selected_cards_index):
        print("\nTry Again")
      else:
        selected_cards_index.append(selected_index)
        selected_cards_name.append(hand[selected_index])

    for i in reversed(selected_cards_index):
        hand.get(i)
        hand_rank.pop(i)

    if(isFirstRound):
      print("\nUser called ",total_selected_cards, " of ", card_rank, "'s")
    else:
      print("\nUser added more ",total_selected_cards, " of ", card_rank, "'s")

  # if agent is playing - Honesty or Bluff
  else:
    # agent choses to be honest
    if(option == 0):
      total_selected_cards = hand_rank.count(card_rank)

      selected_cards_index = []
      selected_cards_name = []

      while len(selected_cards_name) != total_selected_cards:

        indices_to_remove = []

        for index, value in enumerate(hand):
          if value.value == card_rank:
            indices_to_remove.append(index)

        for index in reversed(indices_to_remove):
            selected_cards_index.append(index)
            selected_cards_name.append(hand[index])
            hand.get(index)
            hand_rank.pop(index)

    # agent choses to bluff
    else:
      total_selected_cards = random.randint(1, min(3, len(hand_rank)))

      selected_cards_index = []
      selected_cards_name = []

      while len(selected_cards_name) != total_selected_cards:

        selected_index = random.randint(0,len(hand)-1)
        if(selected_index in selected_cards_index or hand[selected_index].value == card_rank):
          continue
        else:
          selected_cards_index.append(selected_index)
          selected_cards_name.append(hand[selected_index])
          hand.get(selected_index)
          hand_rank.pop(selected_index)


    if(isFirstRound):
      print("\nAgent called ",total_selected_cards, " of ", card_rank, "'s")
    else:
      print("\nAgent added more ",total_selected_cards, " of ", card_rank, "'s")

  # the third argument is isFirstRound
  return selected_cards_name, card_rank, False

In [ ]:
def safe_int_input(prompt):
  while True:
    try:
      return int(input(prompt))
    except ValueError:
      print("Invalid input. Please enter an integer.")

In [ ]:
def printHand(hand):
  print("\nYour hand ( ", len(hand),  " cards ): \n")

  for index, value in enumerate(hand):
    print(index, " -> ", value)

In [ ]:
def choice(isUser, resetRound):
  if(isUser):
    if(resetRound):
      option = 0
    else:
      print("\nYou can chose either one of the following\n0 -> Continue Playing \n1 -> Call Bluff \n2 -> Pass")
      option = safe_int_input("\nEnter your choice: ")
      while(option < 0 or option > 2):
        print("\nWrong Choice")
        option = safe_int_input("Option: ")
  else:
    option = random.randint(0,2)
    while resetRound and (option == 2 or option == 1):
      option = random.randint(0,2)

  # the second argument is resetRound
  return option, False

In [ ]:
def bluff(isUser, user_hand, agent_hand, user_hand_rank, agent_hand_rank, last_played_cards, card_rank, total_cards_round):

  isBluffing = False
  for i in last_played_cards:
    if(str(i.value) != card_rank):
      isBluffing = True
      break

  # If the user calls the bluff & it is a bluff
  if(isBluffing and isUser):
    agent_hand.insert_list(total_cards_round)

    for i in total_cards_round:
      agent_hand_rank.append(i.value)

  # If the user calls the bluff & it is not a bluff
  elif(not isBluffing and isUser):
    user_hand.insert_list(total_cards_round)

    for i in total_cards_round:
      user_hand_rank.append(i.value)

  # If the agent calls the bluff & it is a bluff
  elif(isBluffing and not isUser):
    user_hand.insert_list(total_cards_round)

    for i in total_cards_round:
      user_hand_rank.append(i.value)

  # If the agent calls the bluff & it is not a bluff
  elif(not isBluffing and not isUser):
    agent_hand.insert_list(total_cards_round)

    for i in total_cards_round:
      user_hand_rank.append(i.value)

  return True if isBluffing else False

In [ ]:
# User Starts First

printHand(user_hand)
isFirstRound = True;
selected_cards_name, card_rank, isFirstRound = call(True, user_hand, user_hand_rank, isFirstRound, "")
total_cards_round = selected_cards_name
last_played_cards = selected_cards_name
userPlaying = False
resetRound = False
roundCardRank = card_rank
pass_check = 2

while(len(user_hand) != 0 and len(agent_hand) != 0):


  print("\n-----------------------------------debug------------------------------------")
  print("user_hand_rank: ",user_hand_rank)
  print("agent_hand_rank: ",agent_hand_rank)
  print("-----------------------------------debug------------------------------------")

  # if users play pass consecutively then reset round
  if(pass_check == 0):

    resetRound = True
    isFirstRound = True

    if(userPlaying):
      userPlaying = False

    total_cards_round = []
    pass_check = 2

  # if round resets
  if(resetRound):
    print("\nResetting Round")

  # if it is agents turn
  if(not userPlaying):
    option, resetRound = choice(False, resetRound)

    if(option == 0):
      print("\nAgent chose to continue playing")
      selected_cards_name, card_rank, isFirstRound = call(False, agent_hand, agent_hand_rank ,isFirstRound, roundCardRank)
      total_cards_round = total_cards_round + selected_cards_name
      last_played_cards = selected_cards_name
      userPlaying = True
    elif(option == 1):
      print("\nAgent chose to call bluff")
      bluffCaught = bluff(False, user_hand, agent_hand, user_hand_rank, agent_hand_rank, last_played_cards, card_rank, total_cards_round)
      if(bluffCaught):
        print("\nBluff was caught")
        userPlaying = False
      else:
        print("\nNo Bluff")
        userPlaying = True
      total_cards_round = []
      resetRound = True
      isFirstRound = True
      roundCardRank = ""
      last_played_cards = []
    elif(option == 2):
      print("\nAgent chose to pass")
      userPlaying = True
      pass_check -= 1
      last_played_cards = []

    print("\nPile: ",len(total_cards_round), " & Agent Cards: ",len(agent_hand), " & User Cards: ",len(user_hand))

  # if it is users turn
  else:
    option, resetRound = choice(True, resetRound)
    if(option == 0):
      printHand(user_hand)
      selected_cards_name, card_rank, isFirstRound = call(True, user_hand, user_hand_rank ,isFirstRound, roundCardRank)
      total_cards_round = total_cards_round + selected_cards_name
      last_played_cards = selected_cards_name
      userPlaying = False
    elif(option == 1):
      print("\nUser chose to call bluff")
      bluffCaught = bluff(True, user_hand, agent_hand, user_hand_rank, agent_hand_rank, selected_cards_name, card_rank, total_cards_round)
      if(bluffCaught):
        print("\nBluff was caught")
        userPlaying = True
      else:
        print("\nNo Bluff")
        userPlaying = False
      total_cards_round = []
      last_played_cards = []
      resetRound = True
      isFirstRound = True
    elif(option == 2):
      print("\nUser chose to pass")
      userPlaying = False
      pass_check -= 1
      last_played_cards = []

    print("\nPile: ",len(total_cards_round), " & User Cards: ",len(user_hand), " & Agent Cards: ",len(agent_hand))

  if(len(user_hand) == 0):
    print("\nUser Won")
  elif(len(agent_hand) == 0):
    print("\nAgent Won")


Your hand (  7  cards ): 

0  ->  8 of Clubs
1  ->  3 of Spades
2  ->  Ace of Diamonds
3  ->  8 of Diamonds
4  ->  5 of Diamonds
5  ->  Ace of Clubs
6  ->  Queen of Hearts

Enter the rank of the card (caps): Queen

Enter the total number to call: 1

Enter the index of the cards you want to chose for the call: 6

User called  1  of  Queen 's

-----------------------------------debug------------------------------------
user_hand_rank:  ['8', '3', 'Ace', '8', '5', 'Ace']
agent_hand_rank:  ['King', 'Jack', 'King', '6', '7', '4', '9']
-----------------------------------debug------------------------------------

Agent chose to pass

Pile:  1  & Agent Cards:  7  & User Cards:  6

-----------------------------------debug------------------------------------
user_hand_rank:  ['8', '3', 'Ace', '8', '5', 'Ace']
agent_hand_rank:  ['King', 'Jack', 'King', '6', '7', '4', '9']
-----------------------------------debug------------------------------------

You can chose either one of the following
0 -> 

KeyboardInterrupt: Interrupted by user